# A Deeper Dive into Pandas

I got inspiration for this notebook from this [Medium Article](https://towardsdatascience.com/learn-advanced-features-for-pythons-main-data-analysis-library-in-20-minutes-d0eedd90d086) & this [Github Repo](https://github.com/FBosler/you-datascientist). The Medium has a few other posts that offer introductory information for Pandas, so fee free to check out the series if you want to learn more. 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pytz
import dateutil.parser

In [78]:
# Read in invoices data
invoices = pd.read_csv('invoices.csv') 

# invoices = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/invoices.csv')

invoices.head()

,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Meal Price,Type of Meal,Heroes Adjustment
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22.0,Dinner,False
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314.0,Lunch,False
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00+02:00,['Addie Patino'],438.0,Dinner,False
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690.0,Lunch,False


In [25]:
sales_team = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/sales_team.csv')

In [3]:
print(invoices.shape, 'Rows x Columns \n')
print(invoices.info())

(50017, 9) Rows x Columns 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50017 entries, 0 to 50016
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Order Id           50017 non-null  object 
 1   Date               50017 non-null  object 
 2   Meal Id            50017 non-null  object 
 3   Company Id         50017 non-null  object 
 4   Date of Meal       50017 non-null  object 
 5   Participants       50017 non-null  object 
 6   Meal Price         50017 non-null  float64
 7   Type of Meal       50017 non-null  object 
 8   Heroes Adjustment  50017 non-null  bool   
dtypes: bool(1), float64(1), object(7)
memory usage: 3.1+ MB
None


# Data Cleaning

Notice that the two date columns are not datetime types, instead they are objects. 
Also notice the date column needs to be specified more to Order Date. 

We can specify the datetime type while reading in the data, but for an exercise we will do it now.

## Column Rename

In [4]:
# First, rename the Date Column
invoices.rename(columns = {'Date' : 'Order Date', 
                           'Date of Meal': 'Date of Meal Original'}, 
                inplace = True)

invoices.columns # View the column names

Index(['Order Id', 'Order Date', 'Meal Id', 'Company Id',
       'Date of Meal Original', 'Participants', 'Meal Price', 'Type of Meal',
       'Heroes Adjustment'],
      dtype='object')

## Converting Dates

Order date appears to be just a date, not datetime. So, let's convert that first

In [5]:
# Convert Order Date Column to Datetime format
invoices['Order Date'] = pd.to_datetime(invoices['Order Date'], format = '%Y-%m-%d')

# Check the data type of a column
print(invoices['Order Date'].dtype.name, '\n') 

# Show first value from column
print(invoices['Order Date'][0])

datetime64[ns] 

2016-05-27 00:00:00


If we look at date of meal, it appears to be in UTC format. Let's first convert it to a Datetime column, then we will create a new column for PST

In [6]:
# Note that Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True
invoices['Date of Meal UTC'] = pd.to_datetime(invoices['Date of Meal Original'], utc = True)

# Check the data type of a column
print(invoices['Date of Meal UTC'].dtype.name, '\n')

# Show first value from column
print(invoices['Date of Meal UTC'][0])

datetime64[ns, UTC] 

2016-05-31 05:00:00+00:00


In [7]:
# Convert UTC to PST
invoices['Date of Meal PST'] = invoices['Date of Meal UTC'].dt.tz_convert('US/Pacific')

# Drop the TZ information
invoices['Date of Meal UTC Clean'] = invoices['Date of Meal UTC'].dt.tz_convert(None)

dates_comparison = invoices[['Date of Meal Original', 'Date of Meal UTC', 'Date of Meal UTC Clean', 'Date of Meal PST']]

dates_comparison.head(10)

,Date of Meal Original,Date of Meal UTC,Date of Meal UTC Clean,Date of Meal PST
0,2016-05-31 07:00:00+02:00,2016-05-31 05:00:00+00:00,2016-05-31 05:00:00,2016-05-30 22:00:00-07:00
1,2018-10-01 20:00:00+02:00,2018-10-01 18:00:00+00:00,2018-10-01 18:00:00,2018-10-01 11:00:00-07:00
2,2014-08-23 14:00:00+02:00,2014-08-23 12:00:00+00:00,2014-08-23 12:00:00,2014-08-23 05:00:00-07:00
3,2014-04-07 21:00:00+02:00,2014-04-07 19:00:00+00:00,2014-04-07 19:00:00,2014-04-07 12:00:00-07:00
4,2015-07-27 14:00:00+02:00,2015-07-27 12:00:00+00:00,2015-07-27 12:00:00,2015-07-27 05:00:00-07:00
5,2014-07-17 20:00:00+02:00,2014-07-17 18:00:00+00:00,2014-07-17 18:00:00,2014-07-17 11:00:00-07:00
6,2014-04-30 21:00:00+02:00,2014-04-30 19:00:00+00:00,2014-04-30 19:00:00,2014-04-30 12:00:00-07:00
7,2014-05-07 09:00:00+02:00,2014-05-07 07:00:00+00:00,2014-05-07 07:00:00,2014-05-07 00:00:00-07:00
8,2015-05-18 13:00:00+02:00,2015-05-18 11:00:00+00:00,2015-05-18 11:00:00,2015-05-18 04:00:00-07:00
9,2016-01-14 22:00:00+01:00,2016-01-14 21:00:00+00:00,2016-01-14 21:00:00,2016-01-14 13:00:00-08:00


# Creating Columns

Creating new information from Order Date

In [8]:
# Create a column for order weekday
invoices['Order Weekday'] = invoices['Order Date'].dt.day_name()

# Create a column for order month
invoices['Order Month'] = invoices['Order Date'].dt.month_name()

# Create a column for order year
invoices['Order Year'] = invoices['Order Date'].dt.year

# Create a column for number of days between order and meal
invoices['Order to Meal'] = (invoices['Order Date'] - invoices['Date of Meal UTC Clean']).dt.days

# View the first row
invoices.head(1)

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5


Creating a column to see how far in advance people order

# Filtering & Group By

## Filtering by Categories
### Convert to Categorical 

[See here for the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) on why we convert to categorical. 

In [9]:
# Another way to convert data types
invoices['Type of Meal'] = invoices['Type of Meal'].astype('category')

# Check the data type of a column
print(f"Data Type Name: {invoices['Type of Meal'].dtype.name}") 

# View all categories, the accessor is cat
invoices['Type of Meal'].cat.categories

Data Type Name: category


Index(['Breakfast', 'Dinner', 'Lunch'], dtype='object')

### Option 1

Creating a dataframe for each category.

In [11]:
breakfast_invoices = invoices.loc[invoices['Type of Meal'] == 'Breakfast']
lunch_invoices = invoices.loc[invoices['Type of Meal'] == 'Lunch']
dinner_invoices = invoices.loc[invoices['Type of Meal'] == 'Dinner']

### Option 2

Creating a dictionary where the categories are the keys and the values are the rows

In [12]:
# Create empty dictionary
invoices_dict = {}

# Loop through each category in the Type of Meal column
for category in invoices['Type of Meal'].cat.categories:
    
    # Create a Dictionary Key with the Category, values are where Type of Meal = Category
    invoices_dict[category] = invoices.loc[invoices['Type of Meal'] == category]

In [13]:
# Print the data type
print(type(invoices_dict['Breakfast']))

# View the head
invoices_dict['Breakfast'].head()

<class 'pandas.core.frame.DataFrame'>


,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5
7,FM608JK1N01BPUQN,2014-05-08,E8WJZ1FOSKZD2MJN,36MFTZOYMTAJP1RK,2014-05-07 09:00:00+02:00,['Amanda Knowles' 'Cheryl Feaster' 'Ginger Hoa...,320.0,Breakfast,False,2014-05-07 07:00:00+00:00,2014-05-07 00:00:00-07:00,2014-05-07 07:00:00,Thursday,May,2014,0
11,8RIGCF74GUEQHQEE,2018-07-23,5XK0KTFTD6OAP9ZP,36MFTZOYMTAJP1RK,2018-07-27 08:00:00+02:00,['Amanda Knowles'],210.0,Breakfast,False,2018-07-27 06:00:00+00:00,2018-07-26 23:00:00-07:00,2018-07-27 06:00:00,Monday,July,2018,-5
12,TH60C9D8TPYS7DGG,2016-12-15,KDSMP2VJ22HNEPYF,36MFTZOYMTAJP1RK,2016-12-13 08:00:00+01:00,['Cheryl Feaster' 'Bret Adams' 'Ginger Hoagland'],755.0,Breakfast,False,2016-12-13 07:00:00+00:00,2016-12-12 23:00:00-08:00,2016-12-13 07:00:00,Thursday,December,2016,1
17,KNF3E3QTOQ22J269,2018-06-20,737T2U7604ABDFDF,36MFTZOYMTAJP1RK,2018-06-15 07:00:00+02:00,['Glenn Gould' 'Cheryl Feaster' 'Ginger Hoagla...,475.0,Breakfast,False,2018-06-15 05:00:00+00:00,2018-06-14 22:00:00-07:00,2018-06-15 05:00:00,Wednesday,June,2018,4


## Group By

### Group By Filtering

In [14]:
# Create new subset, grouping by Type of Meal
type_of_meal = invoices.groupby('Type of Meal')

# Show the group Keys
print(type_of_meal.groups.keys(), '\n')

# Show all Groups with data
type_of_meal.groups

dict_keys(['Breakfast', 'Dinner', 'Lunch']) 



{'Breakfast': Int64Index([    0,     7,    11,    12,    17,    21,    22,    25,    26,
                27,
             ...
             49994, 49998, 50001, 50002, 50005, 50008, 50009, 50011, 50012,
             50013],
            dtype='int64', length=16663),
 'Dinner': Int64Index([    1,     3,     5,     6,     9,    10,    16,    23,    29,
                31,
             ...
             49982, 49984, 49987, 49993, 49995, 49996, 50007, 50010, 50014,
             50015],
            dtype='int64', length=16732),
 'Lunch': Int64Index([    2,     4,     8,    13,    14,    15,    18,    19,    20,
                24,
             ...
             49989, 49990, 49992, 49997, 49999, 50000, 50003, 50004, 50006,
             50016],
            dtype='int64', length=16622)}

In [15]:
# Select a specific Group
breakfast_group = type_of_meal.get_group('Breakfast')

# Select unique values from the Meal column
breakfast_group['Type of Meal'].unique()

[Breakfast]
Categories (1, object): [Breakfast]

### Group By Aggregation

In [18]:
# Create the aggregation setup
aggregation = {'Total Orders': pd.NamedAgg(column='Order Id', aggfunc=pd.Series.nunique),
               'Meal Price Max': pd.NamedAgg(column='Meal Price', aggfunc='max'),
               'Meal Price Average': pd.NamedAgg(column='Meal Price', aggfunc='mean') 
              }

# Apply the Group By Aggregation
invoices.groupby("Type of Meal").agg(**aggregation)

,Total Orders,Meal Price Max,Meal Price Average
Type of Meal,,,
Breakfast,16663,1059.0,404.625338
Dinner,16732,1065.0,400.074528
Lunch,16622,1033.0,405.087956


In [19]:
# Group by Type of Meal, & Order year, get nunique values
invoices.groupby(['Type of Meal', 'Order Year'])['Order Id'].nunique()

Type of Meal  Order Year
Breakfast     2014          3361
              2015          3342
              2016          3303
              2017          3298
              2018          3359
Dinner        2014          3388
              2015          3361
              2016          3272
              2017          3269
              2018          3442
Lunch         2014          3320
              2015          3336
              2016          3282
              2017          3370
              2018          3314
Name: Order Id, dtype: int64

In [22]:
# Same function, but with an unstack at the end, DIY Pivot Table
invoices.groupby(['Type of Meal', 'Order Year'])['Order Id'].nunique().unstack()

66.9 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pivot Table

In [23]:
# Same as above, but using pivot table instead
invoices.pivot_table('Order Id', index='Type of Meal', columns='Order Year', aggfunc = lambda x: len(x.unique()))

30 ms ± 943 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Map

pandas.map() is used to map values from two series having one column same. For mapping two series, the last column of the first series should be same as index column of the second series, also the values should be unique.

In [30]:
invoices.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22.0,Dinner,False,2018-10-01 18:00:00+00:00,2018-10-01 11:00:00-07:00,2018-10-01 18:00:00,Thursday,September,2018,-5
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314.0,Lunch,False,2014-08-23 12:00:00+00:00,2014-08-23 05:00:00-07:00,2014-08-23 12:00:00,Sunday,August,2014,0
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00+02:00,['Addie Patino'],438.0,Dinner,False,2014-04-07 19:00:00+00:00,2014-04-07 12:00:00-07:00,2014-04-07 19:00:00,Saturday,April,2014,4
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690.0,Lunch,False,2015-07-27 12:00:00+00:00,2015-07-27 05:00:00-07:00,2015-07-27 12:00:00,Tuesday,July,2015,0


In [28]:
# Create a pandas series where  sales rep is the index, sales rep ID is the value
rep_id = sales_team.set_index('Sales Rep')['Sales Rep Id']

# Same as above, drop the duplicates
rep_id = rep_id.drop_duplicates()

rep_id[:5] # show first 5 values

Sales Rep
Jessie Mcallister    97UNNAT790E0WM4N
Lois Bowers          RRD2R9XMAJDP7TUY
Melvin Dupree        G9T8KNQF8XD2XHTS
Betty Factor         VZV7I6YA8IAX510X
John Rodriguez       HQRE2LG21W2OHCY5
Name: Sales Rep Id, dtype: object

In [53]:
# Create a new column for Rep Id by using the map function for a Pandas Series
invoices_join['Sales Rep Id'] = invoices_join['Sales Rep'].map(rep_id)

invoices_join.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal,Company Name,Sales Rep,Sales Rep Id
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5,Chimera-Chasing Casbah,Jessie Mcallister,97UNNAT790E0WM4N
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22.0,Dinner,False,2018-10-01 18:00:00+00:00,2018-10-01 11:00:00-07:00,2018-10-01 18:00:00,Thursday,September,2018,-5,Chimera-Chasing Casbah,Jessie Mcallister,97UNNAT790E0WM4N
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314.0,Lunch,False,2014-08-23 12:00:00+00:00,2014-08-23 05:00:00-07:00,2014-08-23 12:00:00,Sunday,August,2014,0,Chimera-Chasing Casbah,Jessie Mcallister,97UNNAT790E0WM4N
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00+02:00,['Addie Patino'],438.0,Dinner,False,2014-04-07 19:00:00+00:00,2014-04-07 12:00:00-07:00,2014-04-07 19:00:00,Saturday,April,2014,4,Chimera-Chasing Casbah,Jessie Mcallister,97UNNAT790E0WM4N
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690.0,Lunch,False,2015-07-27 12:00:00+00:00,2015-07-27 05:00:00-07:00,2015-07-27 12:00:00,Tuesday,July,2015,0,Chimera-Chasing Casbah,Jessie Mcallister,97UNNAT790E0WM4N


### Melt

Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’.

In [54]:
company_info.head()

,Company Id,Company Name,Sales Rep
0,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,Jessie Mcallister
1,36MFTZOYMTAJP1RK,Tangential Sheds,Jessie Mcallister
2,H3JRC7XX7WJAD4ZO,Two-Mile Grab,Jessie Mcallister
3,HB25MDZR0MGCQUGX,Three-Men-And-A-Helper Congo'S,Jessie Mcallister
4,7RVA8TIVBLBXMNO4,Biophysical Battleground,Jessie Mcallister


In [55]:
pd.melt(company_info, id_vars = ['Sales Rep'], value_vars = ['Company Name'])

,Sales Rep,variable,value
0,Jessie Mcallister,Company Name,Chimera-Chasing Casbah
1,Jessie Mcallister,Company Name,Tangential Sheds
2,Jessie Mcallister,Company Name,Two-Mile Grab
3,Jessie Mcallister,Company Name,Three-Men-And-A-Helper Congo'S
4,Jessie Mcallister,Company Name,Biophysical Battleground
...,...,...,...
4720,Joel Woodbury,Company Name,Jammed-Together Stable
4721,Joel Woodbury,Company Name,Laureate Barbital
4722,Hazel Dickerson,Company Name,Spotty Adoniram
4723,Hazel Dickerson,Company Name,Physicochemical Impatience


### Stacking & Unstacking

I won't go too in depth with stacking and unstacking, but it's another way to pivot and unpivot your data

In [56]:
# Stacking
company_info.stack()

0     Company Id                LJKS5NK6788CYMUU
      Company Name        Chimera-Chasing Casbah
      Sales Rep                Jessie Mcallister
1     Company Id                36MFTZOYMTAJP1RK
      Company Name              Tangential Sheds
                                 ...            
4723  Company Name    Physicochemical Impatience
      Sales Rep                  Hazel Dickerson
4724  Company Id                E4K99D4JR9E40VE1
      Company Name           Fierce Productivity
      Sales Rep                  Hazel Dickerson
Length: 14175, dtype: object

In [57]:
company_info.unstack()

Company Id  0       LJKS5NK6788CYMUU
            1       36MFTZOYMTAJP1RK
            2       H3JRC7XX7WJAD4ZO
            3       HB25MDZR0MGCQUGX
            4       7RVA8TIVBLBXMNO4
                          ...       
Sales Rep   4720       Joel Woodbury
            4721       Joel Woodbury
            4722     Hazel Dickerson
            4723     Hazel Dickerson
            4724     Hazel Dickerson
Length: 14175, dtype: object

In [58]:
company_info_stacked = company_info.stack()
company_info_stacked.head()

0  Company Id            LJKS5NK6788CYMUU
   Company Name    Chimera-Chasing Casbah
   Sales Rep            Jessie Mcallister
1  Company Id            36MFTZOYMTAJP1RK
   Company Name          Tangential Sheds
dtype: object

In [59]:
company_info_unstacked = company_info_stacked.unstack()
company_info_unstacked.head()

,Company Id,Company Name,Sales Rep
0,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,Jessie Mcallister
1,36MFTZOYMTAJP1RK,Tangential Sheds,Jessie Mcallister
2,H3JRC7XX7WJAD4ZO,Two-Mile Grab,Jessie Mcallister
3,HB25MDZR0MGCQUGX,Three-Men-And-A-Helper Congo'S,Jessie Mcallister
4,7RVA8TIVBLBXMNO4,Biophysical Battleground,Jessie Mcallister


# Concat, Merge, Join

We can use join and merge to combine 2 dataframes.

The join method works best when we are joining dataframes on their indexes (though you can specify another column to join on for the left dataframe).

The merge method is more versatile and allows us to specify columns besides the index to join on for both dataframes. If the index gets reset to a counter post merge, we can use set_index to change it back.

## Concat

In [41]:
# Create Group for Breakfasts
breakfast_meals = type_of_meal.get_group('Breakfast')

# Create Group for Lunch
lunch_meals = type_of_meal.get_group('Lunch')

# Create Group for Dinner
dinner_meals = type_of_meal.get_group('Dinner')

# Concatenate all dataframes into a single dataframe
all_meals = pd.concat([breakfast_meals, lunch_meals, dinner_meals])

all_meals.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5
7,FM608JK1N01BPUQN,2014-05-08,E8WJZ1FOSKZD2MJN,36MFTZOYMTAJP1RK,2014-05-07 09:00:00+02:00,['Amanda Knowles' 'Cheryl Feaster' 'Ginger Hoa...,320.0,Breakfast,False,2014-05-07 07:00:00+00:00,2014-05-07 00:00:00-07:00,2014-05-07 07:00:00,Thursday,May,2014,0
11,8RIGCF74GUEQHQEE,2018-07-23,5XK0KTFTD6OAP9ZP,36MFTZOYMTAJP1RK,2018-07-27 08:00:00+02:00,['Amanda Knowles'],210.0,Breakfast,False,2018-07-27 06:00:00+00:00,2018-07-26 23:00:00-07:00,2018-07-27 06:00:00,Monday,July,2018,-5
12,TH60C9D8TPYS7DGG,2016-12-15,KDSMP2VJ22HNEPYF,36MFTZOYMTAJP1RK,2016-12-13 08:00:00+01:00,['Cheryl Feaster' 'Bret Adams' 'Ginger Hoagland'],755.0,Breakfast,False,2016-12-13 07:00:00+00:00,2016-12-12 23:00:00-08:00,2016-12-13 07:00:00,Thursday,December,2016,1
17,KNF3E3QTOQ22J269,2018-06-20,737T2U7604ABDFDF,36MFTZOYMTAJP1RK,2018-06-15 07:00:00+02:00,['Glenn Gould' 'Cheryl Feaster' 'Ginger Hoagla...,475.0,Breakfast,False,2018-06-15 05:00:00+00:00,2018-06-14 22:00:00-07:00,2018-06-15 05:00:00,Wednesday,June,2018,4


Notice the index is non-sequential. This is because we didn't pass the ignore_index argument. This is how we fix it.

In [42]:
# If True, do not use the index values along the concatenation axis
all_meals = pd.concat([breakfast_meals, lunch_meals, dinner_meals], ignore_index = True)

all_meals.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5
1,FM608JK1N01BPUQN,2014-05-08,E8WJZ1FOSKZD2MJN,36MFTZOYMTAJP1RK,2014-05-07 09:00:00+02:00,['Amanda Knowles' 'Cheryl Feaster' 'Ginger Hoa...,320.0,Breakfast,False,2014-05-07 07:00:00+00:00,2014-05-07 00:00:00-07:00,2014-05-07 07:00:00,Thursday,May,2014,0
2,8RIGCF74GUEQHQEE,2018-07-23,5XK0KTFTD6OAP9ZP,36MFTZOYMTAJP1RK,2018-07-27 08:00:00+02:00,['Amanda Knowles'],210.0,Breakfast,False,2018-07-27 06:00:00+00:00,2018-07-26 23:00:00-07:00,2018-07-27 06:00:00,Monday,July,2018,-5
3,TH60C9D8TPYS7DGG,2016-12-15,KDSMP2VJ22HNEPYF,36MFTZOYMTAJP1RK,2016-12-13 08:00:00+01:00,['Cheryl Feaster' 'Bret Adams' 'Ginger Hoagland'],755.0,Breakfast,False,2016-12-13 07:00:00+00:00,2016-12-12 23:00:00-08:00,2016-12-13 07:00:00,Thursday,December,2016,1
4,KNF3E3QTOQ22J269,2018-06-20,737T2U7604ABDFDF,36MFTZOYMTAJP1RK,2018-06-15 07:00:00+02:00,['Glenn Gould' 'Cheryl Feaster' 'Ginger Hoagla...,475.0,Breakfast,False,2018-06-15 05:00:00+00:00,2018-06-14 22:00:00-07:00,2018-06-15 05:00:00,Wednesday,June,2018,4


## Merge

In [43]:
# Read in invoices data
sales_team = pd.read_csv('sales_team.csv') 

# invoices = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/sales_team.csv')

sales_team.head()

,Sales Rep,Sales Rep Id,Company Name,Company Id
0,Jessie Mcallister,97UNNAT790E0WM4N,Chimera-Chasing Casbah,LJKS5NK6788CYMUU
1,Jessie Mcallister,97UNNAT790E0WM4N,Tangential Sheds,36MFTZOYMTAJP1RK
2,Jessie Mcallister,97UNNAT790E0WM4N,Two-Mile Grab,H3JRC7XX7WJAD4ZO
3,Jessie Mcallister,97UNNAT790E0WM4N,Three-Men-And-A-Helper Congo'S,HB25MDZR0MGCQUGX
4,Jessie Mcallister,97UNNAT790E0WM4N,Biophysical Battleground,7RVA8TIVBLBXMNO4


We are going to get company names, so let's make a subset of the data. Notice that there are duplicate company names, so we need to fix that in our new companies dataset.

In [47]:
# We are going to get company names, so let's make a subset of the data
company_info = sales_team[['Company Id', 'Company Name', 'Sales Rep']]

# There are duplicates in the subset, so let's remove those
company_info = company_info.drop_duplicates().reset_index(drop = True)

company_info.head()

,Company Id,Company Name,Sales Rep
0,LJKS5NK6788CYMUU,Chimera-Chasing Casbah,Jessie Mcallister
1,36MFTZOYMTAJP1RK,Tangential Sheds,Jessie Mcallister
2,H3JRC7XX7WJAD4ZO,Two-Mile Grab,Jessie Mcallister
3,HB25MDZR0MGCQUGX,Three-Men-And-A-Helper Congo'S,Jessie Mcallister
4,7RVA8TIVBLBXMNO4,Biophysical Battleground,Jessie Mcallister


In [49]:
invoices.head(1)

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5


In [50]:
# Merge the dataframes with an inner join on the Company Id column
invoices_merge = pd.merge(left = invoices, right = company_info, how = 'inner', on = 'Company Id')

invoices_merge.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal,Company Name,Sales Rep
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5,Chimera-Chasing Casbah,Jessie Mcallister
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22.0,Dinner,False,2018-10-01 18:00:00+00:00,2018-10-01 11:00:00-07:00,2018-10-01 18:00:00,Thursday,September,2018,-5,Chimera-Chasing Casbah,Jessie Mcallister
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314.0,Lunch,False,2014-08-23 12:00:00+00:00,2014-08-23 05:00:00-07:00,2014-08-23 12:00:00,Sunday,August,2014,0,Chimera-Chasing Casbah,Jessie Mcallister
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00+02:00,['Addie Patino'],438.0,Dinner,False,2014-04-07 19:00:00+00:00,2014-04-07 12:00:00-07:00,2014-04-07 19:00:00,Saturday,April,2014,4,Chimera-Chasing Casbah,Jessie Mcallister
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690.0,Lunch,False,2015-07-27 12:00:00+00:00,2015-07-27 05:00:00-07:00,2015-07-27 12:00:00,Tuesday,July,2015,0,Chimera-Chasing Casbah,Jessie Mcallister


## Join

In [51]:
# Join invoices & companies on Company Id
invoices_join = invoices.join(company_info, on = 'Company Id')

invoices_join.head()

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

Notice how this fails. When we check the documentation for the join function, we see this for the on parameter: 

> Column or index level name(s) in the caller to join on the index in other, otherwise joins index-on-index. If multiple values given, the other DataFrame must have a MultiIndex. Can pass an array as the join key if it is not already contained in the calling DataFrame. Like an Excel VLOOKUP operation.

So, join always uses the index from the other dataframe. So, we need to do this for the join to work

In [52]:
# Join invoices & companies on Company Id, set index to Company Id for companies
invoices_join = invoices.join(company_info.set_index('Company Id'), on = 'Company Id')

invoices_join.head()

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal,Company Name,Sales Rep
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5,Chimera-Chasing Casbah,Jessie Mcallister
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 20:00:00+02:00,['David Bishop'],22.0,Dinner,False,2018-10-01 18:00:00+00:00,2018-10-01 11:00:00-07:00,2018-10-01 18:00:00,Thursday,September,2018,-5,Chimera-Chasing Casbah,Jessie Mcallister
2,041ORQM5OIHTIU6L,2014-08-24,E4UJLQNCI16UX5CS,LJKS5NK6788CYMUU,2014-08-23 14:00:00+02:00,['Karen Stansell'],314.0,Lunch,False,2014-08-23 12:00:00+00:00,2014-08-23 05:00:00-07:00,2014-08-23 12:00:00,Sunday,August,2014,0,Chimera-Chasing Casbah,Jessie Mcallister
3,YT796QI18WNGZ7ZJ,2014-04-12,C9SDFHF7553BE247,LJKS5NK6788CYMUU,2014-04-07 21:00:00+02:00,['Addie Patino'],438.0,Dinner,False,2014-04-07 19:00:00+00:00,2014-04-07 12:00:00-07:00,2014-04-07 19:00:00,Saturday,April,2014,4,Chimera-Chasing Casbah,Jessie Mcallister
4,6YLROQT27B6HRF4E,2015-07-28,48EQXS6IHYNZDDZ5,LJKS5NK6788CYMUU,2015-07-27 14:00:00+02:00,['Addie Patino' 'Susan Guerrero'],690.0,Lunch,False,2015-07-27 12:00:00+00:00,2015-07-27 05:00:00-07:00,2015-07-27 12:00:00,Tuesday,July,2015,0,Chimera-Chasing Casbah,Jessie Mcallister


### Joining Strings

Today I found this [great article about joining strings](https://towardsdatascience.com/do-not-use-to-join-strings-in-python-f89908307273) and figured it would be a great addition to this section. Let's go over how to join strings.

Previously, I showed you how to join strings in a print statement by doing this.

In [60]:
string_1 = 'I love'
string_2 = 'Python'

print(string_1 + ' ' + string_2)

I love Python


Then Eric told us about a better way to join strings, like this

In [61]:
print(f'{string_1} {string_2}')

I love Python


And there's a whole host of other ways to concatenate strings together, as seen in this great [Stack Overflow post](https://stackoverflow.com/questions/15286401/print-multiple-arguments-in-python). But, we're going to show you a more efficient way to concatenate strings so you can add them into your dataframe. First, let's show you the original way

In [62]:
# Define a function
def join_strings(strs):
    # result is an empty string
    result = ''
    
    # loop through each value in the list
    for s in strs:
        
        # result = itself plus a space, plus the value in the list
        result += ' ' + s
     
    # return the result
    return result[1:]

# Define our list of strings
strs = ['Life', 'is', 'short,', 'I', 'use', 'Python']

# Pass the list through the function
join_strings(strs)

'Life is short, I use Python'

Now let's create another function where with the more efficient way to join strings

In [63]:
# Create the function
def join_strings_better(strs): 
    
    # Joing each value in the list with a space after it
    return ' '.join(strs)

# Pass the list through the function
join_strings_better(strs)

'Life is short, I use Python'

Now for fun, let's time the functions to see which is faster

In [64]:
%timeit join_strings(strs)

%timeit join_strings_better(strs)

1.15 µs ± 52.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
268 ns ± 7.58 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Now let's take this function and apply it to a dataframe using breakfast_meals

In [65]:
breakfast_meals.head(1)

,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5


In [66]:
pd.options.mode.chained_assignment = None  # default='warn'

In [73]:
breakfast_meals['Type of Meal'] = breakfast_meals['Type of Meal'].astype(str)

In [75]:
# Method 1
%timeit breakfast_meals["Day Test 1"] = breakfast_meals["Order Weekday"] + ' ' + breakfast_meals["Type of Meal"]

# Method 2  
%timeit breakfast_meals["Day Test 2"] = breakfast_meals[["Order Weekday", "Type of Meal"]].agg(' '.join, axis=1)

# View the head
breakfast_meals.head()

4.08 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
981 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,Order Id,Order Date,Meal Id,Company Id,Date of Meal Original,Participants,Meal Price,Type of Meal,Heroes Adjustment,Date of Meal UTC,Date of Meal PST,Date of Meal UTC Clean,Order Weekday,Order Month,Order Year,Order to Meal,Day Test 1,Day Test 2
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 07:00:00+02:00,['David Bishop'],469.0,Breakfast,False,2016-05-31 05:00:00+00:00,2016-05-30 22:00:00-07:00,2016-05-31 05:00:00,Friday,May,2016,-5,Friday Breakfast,Friday Breakfast
7,FM608JK1N01BPUQN,2014-05-08,E8WJZ1FOSKZD2MJN,36MFTZOYMTAJP1RK,2014-05-07 09:00:00+02:00,['Amanda Knowles' 'Cheryl Feaster' 'Ginger Hoa...,320.0,Breakfast,False,2014-05-07 07:00:00+00:00,2014-05-07 00:00:00-07:00,2014-05-07 07:00:00,Thursday,May,2014,0,Thursday Breakfast,Thursday Breakfast
11,8RIGCF74GUEQHQEE,2018-07-23,5XK0KTFTD6OAP9ZP,36MFTZOYMTAJP1RK,2018-07-27 08:00:00+02:00,['Amanda Knowles'],210.0,Breakfast,False,2018-07-27 06:00:00+00:00,2018-07-26 23:00:00-07:00,2018-07-27 06:00:00,Monday,July,2018,-5,Monday Breakfast,Monday Breakfast
12,TH60C9D8TPYS7DGG,2016-12-15,KDSMP2VJ22HNEPYF,36MFTZOYMTAJP1RK,2016-12-13 08:00:00+01:00,['Cheryl Feaster' 'Bret Adams' 'Ginger Hoagland'],755.0,Breakfast,False,2016-12-13 07:00:00+00:00,2016-12-12 23:00:00-08:00,2016-12-13 07:00:00,Thursday,December,2016,1,Thursday Breakfast,Thursday Breakfast
17,KNF3E3QTOQ22J269,2018-06-20,737T2U7604ABDFDF,36MFTZOYMTAJP1RK,2018-06-15 07:00:00+02:00,['Glenn Gould' 'Cheryl Feaster' 'Ginger Hoagla...,475.0,Breakfast,False,2018-06-15 05:00:00+00:00,2018-06-14 22:00:00-07:00,2018-06-15 05:00:00,Wednesday,June,2018,4,Wednesday Breakfast,Wednesday Breakfast


# Pandas Profiling

Pandas profiling is a package separate from Pandas, but is built on pandas functionality. So you must first install that package

In [76]:
from pandas_profiling import ProfileReport

In [79]:
invoices_profile = ProfileReport(invoices)
invoices_profile.to_file(output_file='invoices_profile.html')

TypeError: concat() got an unexpected keyword argument 'join_axes'